# 7장 K-mer Counting

- K-mer는 길이가 K 개인 substring임.
- K-mer의 빈도 측정은 DNA 서열 데이터 분석에서 중요한 단계임.
- K-mer Counting은 유전자와 전사체의 결합, sequence의 error correction 을 위해서 사용됨.
- K-mer Counting 단순하지만, bigdata의 경우 큰 도전임.
- 수십억의 DNA 서열에서 K-mer Counting은 어려운 문제임.


- 이장의 목표 
    - K > 0 주어졌을때 모든 K-mers을 탐색
    - N > 0 이상의 K-mers 중에서 top N을 찾기
    
    
```

original sequence: CACACACAGT
3-mers:            CAC
                    ACA
                     CAC
                      ACA
                       CAC
                        ACA
                         CAG
                          AGT
                          
original sequence: CACACACAGT
5-mers:            CACAC
                    ACACA
                     CACAC
                      ACACA
                       CACAG
                        ACAGT
```

## Input Data for K-mer Counting

- FASTQ 파일 포맷을 입력값으로 사용함.
```
@EAS54_6_R7_30800
GTTGCTTCCCGCGTGGGTGGGTCGGGG
+EAS54_6_R7
;;;;;;;33;;;;9;7;;.7;393333
```

- FASTQ file format은 첫번째 라인이 @로 시작되고, 세번째 라인은 + 시작, 네번째 라인은 서열의 품질을 표현함.
- 두번째 라인이 우리의 분석 대상임.

## Sample Data for K-mer Counting

K-mer counting을 위한 테스트용 샘플
- E. coli genome from http://bit.ly/e_coli_genome.
- Human genome from http://bit.ly/hum_genome. Note that hg19 means human genome, build 19

K-mer counting은 아래와 같은 질문에 답변을 줄 수 있음.
- What are the top 10 most frequently occurring 9-mers in E. coli?
- What are the top 10 most frequently occurring 9-mers in hg19?
- What are the top 10 most frequently occurring 21-mers in hg19?

##  K-mer Counting의 응용

- reads들간의 misalignment가 sequencing error 인지, 선천적인 차이인지 구분할때. 
- 생물학적으로 중요한 역할을 하는 transposons 같은 반복적인 서열을 찾을때 
- Correcting short-read assembly errors
- Computing metrics such as relatedness(관계성) and unique enough(충분한 고유성) (useful in metagenomic applications)

## K-mer Counting Solution in MapReduce/Hadoop

### The map() Function

![](chap17_01.jpg)

### The reduce() Function

![](chap17_02.jpg)

```
CACACACAGT   의 K = 3


CAC 1
ACA 1
CAC 1
ACA 1
CAC 1
ACA 1
CAG 1
AGT 1


ACA 3
CAC 3
CAG 1
AGT 1
```

## K-mer Counting Solution in Spark

![](chap17_03.jpg)

### Step 3: Create a Spark context Object

In [3]:
from pyspark import SparkContext
sc = SparkContext() 
sc

### Step 4: Broadcast global shared objects

- 여러 노드에서 설정값을 공유하기 위해서 spark에서는 broadcast 라는 공유변수를 지원함.

In [29]:
K = 3  # to find K-mers
N = 5  # to find top N

broadcastK = sc.broadcast(K)
broadcastN = sc.broadcast(N)

### Step 5: Read the FASTQ file from HDFS and create the first RDD

sample.fastq

```
@EAS54_6_R1_2_1_413_324
CCCTTCTTGTCCCCAGCGTTTCTCC
+
;;3;;;;;;;;;;;;7;;;;;;;88
@EAS54_6_R1_2_1_540_792
TTGGCAGGCCAAGGCCGATGGATCA
+
;;;;;;;;;;;7;;;;;-;;;3;83
@EAS54_6_R1_2_1_443_348
GTTGCTTCTGGCGTGGGTGGGGGGG
+EAS54_6_R1_2_1_443_348
;;;;;;;;;;;9;7;;.7;393333
@EAS54_6_R1_2_1_413_324
CCCCCCTTGTCTTCAGCCCTTCTCC
+
;;3;;;;;;;;;;;;7;;;;;;;88
@EAS54_6_R1_2_1_540_792
TTTTCAGGCCAAGGCCGATGGATCA
+
;;;;;;;;;;;7;;;;;-;;;3;83
@EAS54_6_R1_2_1_443_348
GTTGTTTCTGGCGTGGGTGGGGGGG
+EAS54_6_R1_2_1_443_348
;;;;;;;;;;;9;7;;.7;393333
@EAS54_6_R1_2_1_443_348
GTTGTTTCTGGCGTGGGTGGCCCCC
+EAS54_6_R1_2_1_443_348
;;;;;;;;;;;9;7;;.7;393333
```

In [9]:
records = sc.textFile('sample.fastq', 1);

In [ ]:
records.collect()

### Step 6: Filter redundant records

In [18]:
def is_redundant_records( record ) :
    firstChar = record[ 0 : 1 ]
    if  "@" == firstChar  or  "+" == firstChar  or  ";" == firstChar  or  "~" == firstChar  : 
        return False
    else :
        return True

In [19]:
filteredRDD = records.filter( is_redundant_records  ) 

In [ ]:
filteredRDD.collect()

### Step 7: Generate K-mers

In [30]:
def generate_Kmers( sequence )  :
    K = broadcastK.value
    mylist = []
    for i in range( 0,  len(sequence)-K+1 ) :
        kmer = sequence[ i : K + i ] 
        mylist.append( ( kmer, 1 ) )
    return mylist

In [31]:
kmers = filteredRDD.flatMap( generate_Kmers )

In [ ]:
kmers.collect()

### Step 8: Combine/reduce frequent K-mers

In [45]:
kmersGrouped = kmers.reduceByKey( lambda a, b: a + b  )

In [46]:
kmersGrouped.collect()

[(u'CTT', 6),
 (u'ATG', 2),
 (u'AAG', 2),
 (u'ATC', 2),
 (u'AGG', 4),
 (u'CCT', 3),
 (u'AGC', 2),
 (u'CTG', 3),
 (u'CTC', 2),
 (u'CAA', 2),
 (u'CAG', 4),
 (u'CCG', 2),
 (u'CCC', 11),
 (u'GGT', 3),
 (u'TGT', 4),
 (u'CGA', 2),
 (u'CCA', 3),
 (u'TCT', 7),
 (u'GAT', 4),
 (u'TTT', 5),
 (u'TGC', 1),
 (u'GGG', 13),
 (u'GGA', 2),
 (u'TGG', 12),
 (u'GGC', 9),
 (u'TTC', 8),
 (u'TTG', 6),
 (u'CGT', 4),
 (u'TCA', 4),
 (u'GCA', 1),
 (u'GCC', 6),
 (u'GTC', 2),
 (u'GCG', 4),
 (u'GTG', 6),
 (u'GTT', 6),
 (u'GCT', 1),
 (u'TCC', 3)]

### Step 9: Create a local top N for each partition

In [61]:
def getKey(item):
    return item[1]


def local_topN( iter ) :
    N = broadcastN.value
    topN = []
    kmerslist = list(iter)
    for i in xrange(len(kmerslist)):
        tup = kmerslist[i] 
        topN.append( tup )
        if len( topN )  > N :
            topN = sorted( topN, key=getKey, reverse=True)   # 비효율적인 방식임.
            topN.pop()

    return topN

In [62]:
partitions = kmersGrouped.mapPartitions( local_topN )

In [63]:
partitions.collect()

[(u'GGG', 13), (u'TGG', 12), (u'CCC', 11), (u'GGC', 9), (u'TTC', 8)]

### Step 10: Find the final top N

In [64]:
alltopN = partitions.collect()
sorted( alltopN, key=getKey, reverse=True) 

[(u'GGG', 13), (u'TGG', 12), (u'CCC', 11), (u'GGC', 9), (u'TTC', 8)]